# Roma Satellite Imagery Analysis with OpenEO

## Overview

This notebook demonstrates how to retrieve and visualize satellite imagery of Rome using the OpenEO API. We'll use Sentinel-2 data to create a true-color RGB image of the Rome area.

In [1]:
# Import required packages
import openeo
from openeo.api.process import Parameter
from openeo.processes import process, array_create

## Connect to the OpenEO Backend

We connect to the OpenEO backend service and authenticate using OpenID Connect. This establishes a connection to the remote processing service where our satellite data is stored.

In [2]:
# Connect to the back-end

connection = openeo.connect(
    "https://api.explorer.eopf.copernicus.eu/openeo"
).authenticate_oidc_authorization_code()
# connection = openeo.connect(
#     url="http://127.0.0.1:8081/"
# ).authenticate_oidc_authorization_code()

## Define Spatial Extent

We define the geographic boundaries of our area of interest using longitude and latitude coordinates.

In [5]:
# Bretagne with Sentinel-2 L2A data
collection_id = "sentinel-2-l2a"
bounding_box = Parameter(
    "bounding_box",
    description="Bounding box for the area of interest",
    default={
        "west": -5.0,
        "south": 40.0,
        "east": 5.0,
        "north": 50.0,
    },
)
time = Parameter(
    "time",
    description="Time range for the data",
    default=["2026-01-10", "2026-01-20"],
)
bands = Parameter(
    "bands",
    description="Bands to load",
    default=["B04", "B03", "B02"],
)
cloud_cover = Parameter(
    "cloud_cover",
    description="Maximum cloud cover percentage",
    default=20,
)

## Load Sentinel-2 Data

We load Sentinel-2 satellite imagery for our defined area using the `load_collection` process. 
- We select the RGB bands (B04=Red, B03=Green, B02=Blue) for true-color visualization

In [12]:
cube = connection.load_collection(
    collection_id=collection_id,
    spatial_extent=bounding_box,
    temporal_extent=time,
    bands=bands,
    properties={
        "eo:cloud_cover": lambda x: x <= cloud_cover,
    },
)
cube

## Scale Image Values

Raw satellite reflectance values need to be scaled to standard RGB display range (0-255). We define a processing function that:
1. Scales the values from their original range (0-10000) to 0-255
2. Applies truncation to ensure all values are valid integers within range

In [13]:
def _process1(x, context=None):
    data1 = process("linear_scale_range", inputMax=1, inputMin=0, outputMax=255, x=x)
    data2 = process("trunc", x=data1)
    return data2

## Apply Processing and Enhance Visual Appearance

We apply the scaling function to our data cube, then use a color formula to enhance the visual appearance of the image. The color formula applies:
- Gamma correction (RGB 1.5) to adjust brightness and contrast
- Sigmoidal contrast enhancement (RGB 10 0.3) to improve detail visibility
- Saturation adjustment to enhance color vibrancy

In [ ]:
# Reduce temporal dimension by taking the First value
reduced = cube.process(
    "apply_pixel_selection",
    pixel_selection="first",
    data=cube,
)


def _to_rgb(data):
    return array_create([data[0], data[1], data[2]])


rgb = reduced.apply_dimension(dimension="bands", process=_to_rgb)

processed = rgb.apply(process=_process1)
graph = processed.process(
    "color_formula",
    data=processed,
    formula="Gamma RGB 1.5 Sigmoidal RGB 6 0.3 Saturation 1",
)
graph = graph.save_result(format="PNG")

print(graph.to_json())

{
  "process_graph": {
    "loadcollection1": {
      "process_id": "load_collection",
      "arguments": {
        "bands": {
          "from_parameter": "bands"
        },
        "id": "sentinel-2-l2a",
        "properties": {
          "eo:cloud_cover": {
            "process_graph": {
              "lte1": {
                "process_id": "lte",
                "arguments": {
                  "x": {
                    "from_parameter": "value"
                  },
                  "y": {
                    "from_parameter": "cloud_cover"
                  }
                },
                "result": true
              }
            }
          }
        },
        "spatial_extent": {
          "from_parameter": "bounding_box"
        },
        "temporal_extent": {
          "from_parameter": "time"
        }
      }
    },
    "applypixelselection1": {
      "process_id": "apply_pixel_selection",
      "arguments": {
        "data": {
          "from_node": "loadcollection1"

## Create a XYZ Tile Service


In [15]:
# Create XYZ Service
service = connection.create_service(
    {
        "process_graph": graph.flat_graph(),
        "parameters": [
            time.to_dict(),
            bounding_box.to_dict(),
            bands.to_dict(),
            cloud_cover.to_dict(),
        ],
    },
    id="eopf-load-collection-xyz",
    type="XYZ",
    title="Mosaic of Sentinel-2 L2A with cloud filtering",
    description="Mosaic of Sentinel-2 L2A with cloud filtering",
    configuration={
        "tile_size": 256,
        "minzoom": 5,
        "maxzoom": 14,
        "extent": [
            bounding_box.default["west"],
            bounding_box.default["south"],
            bounding_box.default["east"],
            bounding_box.default["north"],
        ],
        "scope": "public",
    },
)

Preflight process graph validation failed: [400] InvalidRequest: 1 validation error:
  {'type': 'missing', 'loc': ('body', 'id'), 'msg': 'Field required', 'input': {'process_graph': {'loadcollection1': {'process_id': 'load_collection', 'arguments': {'bands': {'from_parameter': 'bands'}, 'id': 'sentinel-2-l2a', 'properties': {'eo:cloud_cover': {'process_graph': {'lte1': {'process_id': 'lte', 'arguments': {'x': {'from_parameter': 'value'}, 'y': {'from_parameter': 'cloud_cover'}}, 'result': True}}}}, 'spatial_extent': {'from_parameter': 'bounding_box'}, 'temporal_extent': {'from_parameter': 'time'}}}, 'applypixelselection1': {'process_id': 'apply_pixel_selection', 'arguments': {'data': {'from_node': 'loadcollection1'}, 'pixel_selection': 'first'}}, 'applydimension1': {'process_id': 'apply_dimension', 'arguments': {'data': {'from_node': 'applypixelselection1'}, 'dimension': 'bands', 'process': {'process_graph': {'arrayelement1': {'process_id': 'array_element', 'arguments': {'data': {'from_

## Conclusion

This notebook demonstrates how to use OpenEO to access, process, and visualize satellite imagery for urban areas. The workflow can be extended to include additional analysis, such as:

- Time series analysis to observe changes over multiple dates
- Land cover classification to identify different urban features
- Spectral indices to analyze vegetation, water, or built-up areas
- Image segmentation to extract specific features